# Source Code

In [ ]:
import numpy as np
import random
import json
import math


class HalfSpinRbmSymm():
    '''
    use +1/-1 to encode the spin-1/2 up/down state
    and take the convention: +1(up): [1,0] & -1(down): [0,1]
    with symmetries imposed directly
    '''
    def __init__(self, spin_num, feature_num, order):
        # 'order' is used to control the magnitude of the RBM parameters
        self.spin_num, self.feature_num = spin_num, feature_num
        self.bias_a = self._random_complex(1, order)[0]
        self.bias_b = self._random_complex(feature_num, order)
        self.weight_w = self._random_complex((feature_num, spin_num), order)
        self.symm_group = [np.eye(spin_num, dtype=int)] # at least one identity element 
        
        self.state = np.random.choice([1,-1], spin_num)
        self.template = list(range(spin_num)) # site index: [0, 1, ..., spin_num-1]
        
        
    def add_symm_map(self, symm_map):
        # attention: do not add the identity element!
        self.symm_group.append(symm_map)
        
        
    def set_state(self, state=''):
        if state != '':
            if np.shape(state) == (self.spin_num,):
                self.state = state.copy()
            else:
                print('Size not matched, check the size of the configuration provided.')
        
        
    def measure(self, operator, markov_eq_step=1000, measure_sample_num=2000):
        # calculate <\phi|operator|\phi> / <\phi|\phi>
        for i in range(markov_eq_step):
            self._sampling_once()
        list_op_loc = []
        for i in range(measure_sample_num):
            self._MC_step()
            list_op_loc.append(self._local_observable(operator, self.state))
        return np.mean(list_op_loc)
        
        
    def SR(self, hamiltonian, regularizer=[True, 100, 0.9, 1e-2, 0], train_sample_num=2000):
        # SR process
        list_H_loc = []
        rank = 1 + self.feature_num + self.spin_num*self.feature_num
        dkdk = np.zeros((rank,rank), dtype=np.complex_) # for <\Delta_{k}\Delta_{k'}>
        dk = np.zeros(rank, dtype=np.complex_) # for <\Delta_{k}>
        dkH = np.zeros(rank, dtype=np.complex_) # for <\Delta_{k}H>
        # order convention: [w, b, a]
            
        for i in range(train_sample_num):
            self._MC_step()
            list_H_loc.append(self._local_observable(hamiltonian, self.state))
            configs = self._generate_equiv_configs(self.state)
            tanh_effective_angles = np.tanh(self.bias_b + np.dot(configs, self.weight_w.T))
            phi_a = np.sum(configs)
            phi_b = np.sum(tanh_effective_angles, axis=0)
            phi_w = np.dot(tanh_effective_angles.T, configs)
            phi_parameters = np.concatenate((phi_w, phi_b, phi_a), axis=None)
            dkdk += np.outer(phi_parameters, phi_parameters)
            dk += phi_parameters
            dkH += phi_parameters * list_H_loc[-1]
            
        dkdk = dkdk / train_sample_num
        dk = dk / train_sample_num
        dkH = dkH / train_sample_num
            
        # np.real() can be applied, yet we did not
        # Reason: the imaginary part can show the goodness of the sampling
        ave_H = np.mean(list_H_loc) 
            
        matrix_S = dkdk - np.outer(dk, dk)
        vector_f = ave_H * dk - dkH
        if regularizer[0] == True: 
            regular_coeff = max(regularizer[1]*pow(regularizer[2],regularizer[4]), regularizer[3]) 
            matrix_invS = np.linalg.pinv(matrix_S+regular_coeff*np.identity(rank)) # regularization
        else:
            matrix_invS = np.linalg.pinv(matrix_S)
        dev_parameters = np.inner(matrix_invS, vector_f)
        
        return ave_H, dev_parameters
        
        
    def GS(self, hamiltonian, learning_rate=0.1, decay_rate=1, 
           sr_regularizer=[True, 100, 0.9, 1e-2, 0], markov_eq_step=1000, 
           train_sample_num=2000, measure_sample_num=2000, epoch=1, obs='', filename=''):
        # ground state solver
        if obs != '':
            obs_flag = 1
        else:
            obs_flag = 0
            
        if filename == '':
            filename = 'output.json'
            
        data_all = {"iteration": [], "energy": {"real": [], "imag": []}}
        if obs_flag == 1:
            for key in obs.keys():
                data_all[key] = {"real": [], "imag": []}
        
        for epo in range(epoch):
            for i in range(markov_eq_step): # reach the equilibrium stage
                self._sampling_once()
                
            # observables
            data_all["iteration"].append(epo)
            if obs_flag == 1:
                for key in obs.keys():
                    result = self.measure(obs[key], markov_eq_step=0, measure_sample_num=measure_sample_num)
                    data_all[key]["real"].append(result.real)
                    data_all[key]["imag"].append(result.imag)
                    
            sr_regularizer[-1] = epo
            ave_H, dev_parameters = self.SR(hamiltonian, 
                                        regularizer=sr_regularizer, train_sample_num=train_sample_num)
            
            rank = 1 + self.feature_num + self.spin_num*self.feature_num
            self.weight_w += learning_rate * pow(decay_rate, epo) * np.reshape(
                dev_parameters[0:self.spin_num*self.feature_num], (self.feature_num, self.spin_num))
            self.bias_b += learning_rate * pow(decay_rate, epo) * dev_parameters[self.spin_num*self.feature_num:-1]
            self.bias_a += learning_rate * pow(decay_rate, epo) * dev_parameters[-1]
            
            data_all["energy"]["real"].append(ave_H.real)
            data_all["energy"]["imag"].append(ave_H.imag)
            ge = ave_H / self.spin_num
            print("epoch %s, E/N= %.8f(%.1ej)" %(epo+1, np.real(ge), np.imag(ge)))
                    
        # save data
        with open(filename, "w") as save_file:
            json.dump(data_all, save_file)
            
            
    def TE(self, hamiltonian, time_step=0.01, markov_eq_step=1000, 
           train_sample_num=2000, measure_sample_num=2000, epoch=1, obs='', filename=''):
        # real-time evolution
        if obs != '':
            obs_flag = 1
        else:
            obs_flag = 0
            
        if filename == '':
            filename = 'output.json'
            
        data_all = {"time": [], "energy": {"real": [], "imag": []}}
        if obs_flag == 1:
            for key in obs.keys():
                data_all[key] = {"real": [], "imag": []}
        
        for epo in range(epoch):
            for i in range(markov_eq_step): # reach the equilibrium stage
                self._sampling_once()
                
            # observables
            data_all["time"].append(epo*time_step)
            if obs_flag == 1:
                for key in obs.keys():
                    result = self.measure(obs[key], markov_eq_step=0, measure_sample_num=measure_sample_num)
                    data_all[key]["real"].append(result.real)
                    data_all[key]["imag"].append(result.imag)
            
            ave_H, dev_parameters = self.SR(hamiltonian, regularizer=[False], train_sample_num=train_sample_num)
            
            rank = 1 + self.feature_num + self.spin_num*self.feature_num
            self.weight_w += 1.j * time_step * np.reshape(
                dev_parameters[0:self.spin_num*self.feature_num], (self.feature_num, self.spin_num))
            self.bias_b += 1.j * time_step * dev_parameters[self.spin_num*self.feature_num:-1]
            self.bias_a += 1.j * time_step * dev_parameters[-1]
            
            data_all["energy"]["real"].append(ave_H.real)
            data_all["energy"]["imag"].append(ave_H.imag)
            ge = ave_H / self.spin_num
            print("time %s, E/N= %.8f(%.1ej)" %(epo*time_step, np.real(ge), np.imag(ge)))
                    
        # save data
        with open(filename, "w") as save_file:
            json.dump(data_all, save_file)
        
        
    def _MC_step(self):
        # perform one Monte Carlo Step
        for i in range(self.spin_num):
            self._sampling_once()

    
    def _sampling_once(self):
        # perform one markov-chain sampling step
        phi_old = self._wf(self.state)
        site = random.choice(self.template) # choose one site randomly
        self.state[site] *= -1 # spin flip
        phi_new = self._wf(self.state)
        prob = np.real( phi_new*np.conj(phi_new) / (phi_old*np.conj(phi_old)) ) # real number
        if random.uniform(0,1) < prob:
            return 0 # accepted
        else:
            self.state[site] *= -1 # spin flip back
            return 1 # rejected 
        
        
    def _local_observable(self, observable, state):
        # obey Python convention: index begins with 0
        # 'observable' is a list storing the local form of an operator, e.g. Hamiltonian
        # in general, local observable should be a complex number!!!
        # for Hermitian operator, the average over 'state' should be real;
        # for non-Hermitian operator, the average would not be necessarily real!
        # 'observable' example: for 2-site Ising model
        # sx = [[0,1],[1,0]], sz = [[1,0],[0,-1]]
        # [ [[0], -sx], [[1], -sx], [[0,1], -np.kron(sz,sz)] ] is a legal 'observable'
        obs_loc = 0.+0.j
        phi_state = self._wf(state)
        for term in observable:
            pos_list = term[0]
            local_degree = len(pos_list) # how many sites are involved in this term
            operator = term[1]
            config = [state[pos] for pos in pos_list] # configuration of the related sites
            row = self._configuration_index(config) # configuration encoding(index)
            eles = operator[row] # only the 'row'-th row of the operator contributes
            for i in range(len(eles)):
                config_prime = self._index_configuration(i, local_degree)
                state_prime = state.copy()
                for k in range(local_degree):
                    state_prime[pos_list[k]] = config_prime[k]
                phi_state_prime = self._wf(state_prime)
                obs_loc += phi_state_prime * eles[i]
        return obs_loc / phi_state
        
        
    def _configuration_index(self, config):
        # 'config' is a list of +1/-1 storing the local configuration
        # this function gives the index of the configuration in the local Hilbert space
        # e.g. for 3-site local Hilbert sapce, index list = [0, 1, 2, ..., 7] for total 8 configurations
        # [-1,1,1] -> 4; [-1,-1,1] -> 6; [1,-1,1] -> 2; [1,1,1] -> 0; [-1,-1,-1] -> 7
        temp = (1+np.array(config))/2 # +1/-1 convert to +1/0 to use binary (e.g. 10101, 0101, 101, etc)
        temp = ''.join(str(int(1-e)) for e in temp) # exchange 1/0 to meet the convention of spin physics
        return int(temp, 2)
        
        
    def _index_configuration(self, pos, length):
        # the inverse of the function self._configuration_index()
        # 'length' is the site number involved in the local Hilbert space
        temp = bin(pos)[2:].zfill(length)
        config = []
        for ele in temp:
            config.append( int(2*(1-int(ele))-1) )
        return config
        
        
    def _wf(self, state):
        # given one of the spin configurations \vec{\sigma}
        # return the corresponding wave function \phi(\vec{sigma})
        configs = self._generate_equiv_configs(state)
        return np.exp(self.bias_a*np.sum(configs)) * np.prod(2*
                np.cosh(self.bias_b+np.dot(configs, self.weight_w.T)))
        
        
    def _effective_angles(self, state):
        # the effective angle is defined as: b^{f} + W^{f}*\vec{\sigma}^{s}
        # the final result will be a matrix: Ns * Nf
        # Nf: # of feature; Ns: # of symmetry elements
        return self.bias_b + np.dot(self._generate_equiv_configs(state), self.weight_w.T)
        
        
    def _generate_equiv_configs(self, spin_config):
        # given a specific spin configuration, 
        # use the symmetry group defined to generate 
        # all equivalent spin configurations
        return np.array([np.inner(symm_map, spin_config) for symm_map in self.symm_group])
        

    def _random_complex(self, size, order):
        # used to generate a complex number array randomly
        # 'size' can be an integer or a tuple of integers, e.g. 4 or (2,3)
        real_part = (np.random.random(size)-0.5) * pow(10, order)
        imag_part = (np.random.random(size)-0.5) * pow(10, order)
        return real_part+1.j*imag_part

# A Simple Example of Quantum Quench Dynamics

## Ising Model with two sites

In [ ]:
# time evolution test (Ising model)
spin_num = 2
feature_num = 1
hi = 0.5
hf = 1.
order = -2

# Hamiltonian and Observables
Hi = []
Hf = []
mx = []
sx = np.array([[0,1],[1,0]])
sz = np.array([[1,0],[0,-1]])
for i in range(spin_num):
    Hi.append([[i],-hi*sx])
    Hf.append([[i],-hf*sx])
    mx.append([[i],sx])
    Hi.append([[i,(i+1)%spin_num],-np.kron(sz,sz)])
    Hf.append([[i,(i+1)%spin_num],-np.kron(sz,sz)])
    
# define the symmetry-version of RBM
model = HalfSpinRbmSymm(spin_num,feature_num,order)

# impose symmetry
symm_map = np.eye(spin_num, dtype=int)
generator = [symm_map[-1]]
for i in range(spin_num-1):
    generator.append(symm_map[i])
for i in range(spin_num-1):
    symm_map = np.dot(generator, symm_map)
    model.add_symm_map(symm_map)
    
model.GS(hamiltonian=Hi, learning_rate=0.05, decay_rate=1., sr_regularizer=[True, 0.1, 0.9, 1e-8, 0], markov_eq_step=400, train_sample_num=800, epoch=100)
model.TE(hamiltonian=Hf, time_step=0.001, markov_eq_step=500, train_sample_num=1000, measure_sample_num=5000, obs={"mx": mx}, filename="mx_te2_ising2_symm.json", epoch=2001)

In [ ]:
### Compare with the exact solution

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

spin_num = 2

time_list = []
mx_list = []
with open("mx_te2_ising2_symm.json", "r") as mx_file:
    data = json.load(mx_file)
time_list = data["time"]
mx_list = data["mx"]["real"]
mx_list = np.array(mx_list) / spin_num

time_exact = []
mx_exact = []
with open("exact_mx_ising2.json") as exact_file:
    data_exact = json.load(exact_file)
data_exact = np.array(data_exact).T
time_exact = data_exact[0]
mx_exact = data_exact[1]

plt.plot(time_list, mx_list, label="NNQS result")
plt.plot(time_exact, mx_exact, label="Exact result", linestyle="dashed")
plt.legend()
plt.xlim(0,2)
plt.xlabel("time")
plt.ylabel("$<\sigma_x>$")
plt.title("Quantum Quench Dynamics of Ising Model ($h_i=0.5$ to $h_f=1$)")
plt.show()

## Ising Model with four sites

In [ ]:
# time evolution test (Ising model)
spin_num = 4
feature_num = 1
hi = 0.5
hf = 1.
order = -2

# Hamiltonian and Observables
Hi = []
Hf = []
mx = []
sx = np.array([[0,1],[1,0]])
sz = np.array([[1,0],[0,-1]])
for i in range(spin_num):
    Hi.append([[i],-hi*sx])
    Hf.append([[i],-hf*sx])
    mx.append([[i],sx])
    Hi.append([[i,(i+1)%spin_num],-np.kron(sz,sz)])
    Hf.append([[i,(i+1)%spin_num],-np.kron(sz,sz)])
    
# define the symmetry-version of RBM
model = HalfSpinRbmSymm(spin_num,feature_num,order)

# impose symmetry
symm_map = np.eye(spin_num, dtype=int)
generator = [symm_map[-1]]
for i in range(spin_num-1):
    generator.append(symm_map[i])
for i in range(spin_num-1):
    symm_map = np.dot(generator, symm_map)
    model.add_symm_map(symm_map)
    
model.GS(hamiltonian=Hi, learning_rate=0.05, decay_rate=1., sr_regularizer=[True, 0.1, 0.9, 1e-8, 0], markov_eq_step=400, train_sample_num=800, epoch=100)
model.TE(hamiltonian=Hf, time_step=0.001, markov_eq_step=500, train_sample_num=1000, measure_sample_num=5000, obs={"mx": mx}, filename="mx_te2_ising4_symm.json", epoch=2001)

In [ ]:
### Compare with the exact solution

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

spin_num = 4

time_list = []
mx_list = []
with open("mx_te2_ising4_symm.json", "r") as mx_file:
    data = json.load(mx_file)
time_list = data["time"]
mx_list = data["mx"]["real"]
mx_list = np.array(mx_list) / spin_num

time_exact = []
mx_exact = []
with open("exact_mx_ising4.json") as exact_file:
    data_exact = json.load(exact_file)
data_exact = np.array(data_exact).T
time_exact = data_exact[0]
mx_exact = data_exact[1]

plt.plot(time_list, mx_list, label="NNQS result")
plt.plot(time_exact, mx_exact, label="Exact result", linestyle="dashed")
plt.legend()
plt.xlim(0,2)
plt.xlabel("time")
plt.ylabel("$<\sigma_x>$")
plt.title("Quantum Quench Dynamics of Ising Model ($h_i=0.5$ to $h_f=1$)")
plt.show()

# Critical Point Test

In [ ]:
# test the critical point (Ising model)
spin_num = 24
feature_num = 2
h = 1.
order = -1

# Hamiltonian and Observables
H = []
sx = np.array([[0,1],[1,0]])
sz = np.array([[1,0],[0,-1]])
for i in range(spin_num):
    H.append([[i],-h*sx])
    H.append([[i,(i+1)%spin_num],-np.kron(sz,sz)])
    
# define the symmetry-version of RBM
model = HalfSpinRbmSymm(spin_num,feature_num,order)

# impose symmetry
symm_map = np.eye(spin_num, dtype=int)
generator = [symm_map[-1]]
for i in range(spin_num-1):
    generator.append(symm_map[i])
for i in range(spin_num-1):
    symm_map = np.dot(generator, symm_map)
    model.add_symm_map(symm_map)
    
model.GS(hamiltonian=H, learning_rate=0.05, decay_rate=0.98, sr_regularizer=[True, 0.1, 0.9, 1e-4, 0], markov_eq_step=800, train_sample_num=1600, epoch=100)
# DMRG bond 100: e(4)=-1.30656296487638; e(8)=-1.28145772387075; e(12)=-1.2768829292567; e(16)=-1.27528715467225
# DMRG bond 100: e(24)=-1.27414902488981; e(36)=-1.27364364589163; e(50)=-1.27344900835956; e(72)=-1.27334055313453
# RBM feature number 2: e(4|2)=-1.30669524; e(8|2)=-1.28107594; e(12|2)=-1.27639132; e(16|2)=-1.27396333;
# RBM feature number 2/4: e(24|2)=-1.27168774; e(36|2)=-1.27178373; e(50|2)=-1.27094075; e(72|2)= Orz.